In [ ]:
import torch

torch.manual_seed(42)
torch.cuda.is_available()

In [ ]:
import pandas as pd
df = pd.read_csv('sample_data/fashion-mnist_train.csv')
df.shape

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image
import numpy as np

class CustomDataset:
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx].reshape(28, 28)
        image = np.uint8(image)
        image = np.stack([image]*3, axis=-1)
        # (a, b, c) -> (c, a, b)
        image = Image.fromarray(image)
        image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

In [ ]:
from torch.utils.data import DataLoader
from torch import nn

train_dataset = CustomDataset(X_train, y_train, transform=custom_transform)
test_dataset = CustomDataset(X_test, y_test, transform=custom_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)
print(vgg16)

In [ ]:
vgg16.features, vgg16.classifier

In [ ]:
import torch.nn as nn

for param in vgg16.features.parameters():
    param.requires_grad = False

vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

vgg16 = vgg16.to("cuda")

In [ ]:
learning_rate = 0.0001
epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
        outputs = vgg16(batch_features)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()
    avg_loss = total_epoch_loss/len(train_loader)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}")

In [ ]:
vgg16.eval()
total = correct = 0

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
        outputs = vgg16(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    accuracy = correct/total
print(accuracy)

In [ ]:
vgg16.eval()
total = correct = 0

with torch.no_grad():
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
        outputs = vgg16(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
    accuracy = correct/total
print(accuracy)